# Zero-Shot Learning

In [ ]:
!pip install transformers==3.1.0

     |████████████████████████████████| 890kB 5.5MB/s 
     |████████████████████████████████| 3.0MB 13.2MB/s 
     |████████████████████████████████| 1.2MB 34.8MB/s 
     |████████████████████████████████| 890kB 40.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=0bf63e0c41c7e8ed864da32aee9a1b11641d37b549bd36d129802975634ea844
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("zero-shot-classification")

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We can use this pipeline by passing in a sequence and a list of candidate labels. The pipeline assumes by default that only one of the candidate labels is true, returning a list of scores for each label which add up to 1.

In [ ]:
sequence = "How is the weather today?"
candidate_labels = ["climate", "environment", "economics"]

classifier(sequence, candidate_labels)

{'labels': ['environment', 'climate', 'economics'],
 'scores': [0.5216448307037354, 0.46808499097824097, 0.010270169004797935],
 'sequence': 'How is the weather today?'}

To do multi-class classification, simply pass `multi_class=True`. In this case, the scores will be independent, but each will fall between 0 and 1.

In [ ]:
sequence = "How is the weather today?"
candidate_labels = ["climate", "environment", "economics", "elections"]

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['climate', 'environment', 'economics', 'elections'],
 'scores': [0.9449107646942139,
  0.9024710655212402,
  0.00020360561029519886,
  0.00011781518696807325],
 'sequence': 'How is the weather today?'}

Here's an example of sentiment classification: 

In [ ]:
sequence = "I hated to visit there. Doesn't recommend"
candidate_labels = ["positive", "negative"]

classifier(sequence, candidate_labels)

{'labels': ['negative', 'positive'],
 'scores': [0.9970691204071045, 0.002930899616330862],
 'sequence': "I hated to visit there. Doesn't recommend"}

So how does this method work?

The underlying model is trained on the task of Natural Language Inference (NLI), which takes in two sequences and determines whether they contradict each other, entail each other, or neither.

This can be adapted to the task of zero-shot classification by treating the sequence which we want to classify as one NLI sequence (called the premise) and turning a candidate label into the other (the hypothesis). If the model predicts that the constructed premise _entails_ the hypothesis, then we can take that as a prediction that the label applies to the text. Check out [this blog post](https://joeddav.github.io/blog/2020/05/29/ZSL.html) for a more detailed explanation.

By default, the pipeline turns labels into hypotheses with the template `This example is {class_name}.`. This works well in many settings, but you can also customize this for your specific setting. Let's add another review to our above sentiment classification example that's a bit more challenging:

In [ ]:
sequences = [
    "I hated to visit there. Doesn't recommend",
    "It was a good experience overall, But could have been better"
]
candidate_labels = ["positive", "negative"]

classifier(sequences, candidate_labels)

[{'labels': ['negative', 'positive'],
  'scores': [0.9970691204071045, 0.002930892864242196],
  'sequence': "I hated to visit there. Doesn't recommend"},
 {'labels': ['positive', 'negative'],
  'scores': [0.9715092182159424, 0.028490804135799408],
  'sequence': 'It was a good experience overall, But could have been better'}]

The second example is a bit harder. Let's see if we can improve the results by using a hypothesis template which is more specific to the setting of review sentiment analysis. Instead of the default, `This example is {}.`, we'll use, `The sentiment of this review is {}.` (where `{}` is replaced with the candidate class name)

In [ ]:
sequences = [
    "I hated to visit there. Doesn't recommend",
    "It was a good experience overall, But could have been better"
]
candidate_labels = ["positive", "negative"]
hypothesis_template = "The sentiment of this review is {}."

classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)

[{'labels': ['negative', 'positive'],
  'scores': [0.9890093207359314, 0.010990672744810581],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['positive', 'negative'],
  'scores': [0.9581228494644165, 0.0418771356344223],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

By providing a more precise hypothesis template, we are able to see a more accurate classification of the second review.

> Note that sentiment classification is used here just as an illustrative example. The [Hugging Face Model Hub](https://huggingface.co/models?filter=text-classification) has a number of models trained specifically on sentiment tasks which can be used instead.